# OpenFOAM Log Viewer

In [ ]:
import numpy as np
import os
import requests
import subprocess
import tempfile

import ipywidgets as widgets
import matplotlib.pyplot as plt
from ipyfilechooser import FileChooser

class FoamLogWidget:
    def __init__(self):
        self.logfile = None
        self.foamlog_cmd = "foamlog"
        self.data = {}
        self.install_foamlog(os.path.join(os.environ["HOME"], "bin"))
        self.file_selector_widget = None
        self.metrics_widget = None
        self.output_widget = None
        self.limit_residuals_widget = None
        self.separate_plots_widget = None
        self.width_widget = None
        self.height_widget = None

    def install_foamlog(self, install_dir):
        foamlog = "https://raw.githubusercontent.com/OpenFOAM/OpenFOAM-dev/master/bin/foamLog"
        foamlog_db = "https://raw.githubusercontent.com/OpenFOAM/OpenFOAM-dev/master/bin/tools/foamLog.db"
        os.makedirs(os.path.join(install_dir, "tools"), exist_ok = True)
        r = requests.get(foamlog)
        self.foamlog_cmd = os.path.join(install_dir, "foamlog")
        with open(self.foamlog_cmd, "w") as f:
            f.write(r.text)
        os.chmod(self.foamlog_cmd, 0o755)
        r = requests.get(foamlog_db)
        with open(os.path.join(install_dir, "tools", "foamlog.db"), "w") as f:
            f.write(r.text)
        
    def read(self):
        cwd = os.getcwd()
        fname = os.path.abspath(self.logfile)
        self.data = {}
        with tempfile.TemporaryDirectory() as tmpdir:
            os.chdir(tmpdir)
            subprocess.call([self.foamlog_cmd, fname], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
            logdir = os.path.join(tmpdir, "logs")
            os.chdir(logdir)
            for f in os.listdir(logdir):
                if f == "foamLog.awk" or f == "foamlog.awk":
                    continue
                self.data[f] = np.genfromtxt(f, delimiter="\t", skip_header=0, filling_values=0)
        os.chdir(cwd)
    
    def _plot_residuals(self, w):
        self.output_widget.clear_output(wait=True)
        if len(self.metrics_widget.value) == 0:
            return
        with self.output_widget:
            lim = self.limit_residuals_widget.value
            if self.separate_plots_widget.value == True and len(self.metrics_widget.value) > 1:
                fig, ax = plt.subplots(nrows=len(self.metrics_widget.value), ncols=1, figsize=(self.width_widget.value, self.height_widget.value))
                for i, m in enumerate(self.metrics_widget.value):
                    ax[i].plot(self.data[m][-lim:,0], self.data[m][-lim:,1], label=m)
                    ax[i].set(xlabel="Iteration")
                    ax[i].legend()
                    ax[i].grid()
            else:
                fig, ax = plt.subplots(figsize=(self.width_widget.value, self.height_widget.value))
                for m in self.metrics_widget.value:
                    ax.plot(self.data[m][-lim:,0], self.data[m][-lim:,1], label=m)
                ax.legend()
                ax.grid()
            plt.show()
            
    def _open_logfile(self, w):
        self.logfile = self.file_selector_widget.selected
        self._refresh(w)
    
    def _refresh(self, w):
        self.read()        
        with self.metrics_widget.hold_trait_notifications():
            old_values = self.metrics_widget.value
            self.metrics_widget.options = self.data.keys()
            new_values = list(set(old_values) & set(self.data.keys()))
        self.metrics_widget.value = list(set(old_values) & set(self.data.keys()))
    
    def display(self):
        self.file_selector_widget = FileChooser(os.environ["HOME"])
        self.file_selector_widget.title = "Select OpenFOAM log file"
        self.file_selector_widget.show_hidden = False
        self.file_selector_widget.filter_patter = "log.*"
        self.file_selector_widget.dir_icon = "/"
        self.file_selector_widget.dir_icon_append = True
        self.file_selector_widget.show_only_dirs = False
        self.file_selector_widget.register_callback(self._open_logfile)
        
        self.separate_plots_widget = widgets.Checkbox(description="Separate plots", value=False)
        self.width_widget = widgets.IntSlider(description="Width", min=4, max=20, value=10)
        self.height_widget = widgets.IntSlider(description="Height", min=2, max=20, value=6)
        self.limit_residuals_widget = widgets.IntSlider(description="Iterations", min=100, max=10000, value=1000, step=50)
        self.metrics_widget = widgets.SelectMultiple(options=self.data.keys(), value=[], rows=10, description='Metrics')
        self.output_widget = widgets.Output()
        refresh_button = widgets.Button(description="Refresh")
        refresh_button.on_click(self._refresh)

        vbox = widgets.VBox([self.separate_plots_widget, self.width_widget, self.height_widget, self.limit_residuals_widget])
        hbox = widgets.HBox([self.metrics_widget, vbox])
        vbox_main = widgets.VBox([self.file_selector_widget, hbox, refresh_button, self.output_widget])
        display(vbox_main)
        
        self.separate_plots_widget.observe(self._plot_residuals, names="value")
        self.width_widget.observe(self._plot_residuals, names="value")
        self.height_widget.observe(self._plot_residuals, names="value")
        self.limit_residuals_widget.observe(self._plot_residuals, names="value")
        self.metrics_widget.observe(self._plot_residuals, names="value")

w = FoamLogWidget()
w.display()